In [21]:
# EXERCISE 1: Product Pricing Manager

import logging
import sys

logging.basicConfig(
    level = logging.INFO,
    format = '%(message)s'
)

#Category-discounts

category_discounts = {"electronics" : 10, "clothing" : 15, "books" : 5, "home" : 12}

#Tier-discounts

tier_discounts = {"premium" : 5, "standard" : 0, "budget" : 2}

def main():
    output_file = "pricing_report.txt"
    try:
        with open("products.txt", "r") as f:
            products_list = f.read()
            print("List of products loaded successfully")
    except FileNotFoundError:
        logging.error("Error: products.txt not found. Please check the location of the file.")
        sys.exit(1)

    lines = products_list.splitlines()

    report_lines = []
    total_products = 0
    total_discount_percent = 0
    total_discount_amount = 0

#Table header

    header = (
        f"{'Product Name':<30}"
        f"{'Base Price':>12}"
        f"{'Discount %':>12}"
        f"{'Discount Amount':>12}"
        f"{'Final Price':>12}"
    )

    separator = "-" * len(header)
    report_lines.append(header)
    report_lines.append(separator)

#Process products
    for line_number, line in enumerate(lines, start = 1):
        line = line.strip()
        if line == "":
            continue
        try:
            name, old_price, category, tier = line.split(",")
        except ValueError:
            logging.warning(f"Skipping line {line_number}: format should be 4 values.")
            continue

            
        name = name.strip()
        category = category.strip().lower()
        tier = tier.strip().lower()

#Convert price
        try:
            price = float(old_price)
        except ValueError:
            logging.warning(f"Skipping line {line_number}: '{old_price}' is not a number.")
            continue

#Check category & tier
        if category not in category_discounts:
            logging.warning(f"Skipping line {line_number}: unknown category '{category}'")
            continue
        if tier not in tier_discounts:
            logging.warning(f"Skipping line {line_number}; unknown tier '{tier}'")
            continue

#Calculate discount
        category_disc = category_discounts[category]
        tier_disc = tier_discounts[tier]
        total_disc = category_disc + tier_disc

        disc_amount = (total_disc / 100) * price
        final_price = price - disc_amount

        report_lines.append(
            f"{name:<30}"
            f"{price:>12.2f}"
            f"{total_disc:>12.2f}"
            f"{disc_amount:>12.2f}"
            f"{final_price:>12.2f}"
        )
        total_products += 1
        total_discount_percent += total_disc
        total_discount_amount += disc_amount

#Report file
    try:
        with open(output_file, "w") as f:
            f.write("PRICING REPORT\n\n")
    
            for line in report_lines:
                f.write(line+ "\n")

            f.write("\nSUMMARY\n")
            f.write(f"Total Products: {total_products} \n")

            if total_products > 0:
                avg_disc = total_discount_percent / total_products
                f.write(f"Average Discount: {avg_disc:.2f}%\n")
                f.write(f"Total Discount Amount: ${total_discount_amount:.2f}\n")

    except PermissionError:
        logging.error("Error: Cannot write to pricing_report.txt")
        sys.exit(1)


#Print summary to console
    print("\n=== SUMMARY (Console Output) ===")
    print(f"Total Products Processed: {total_products}")

    if total_products > 0:
        avg_disc = total_discount_percent / total_products
        print(f"Average Discount Applied: {avg_disc:.2f}%")

    print("Report saved as: pricing_report.txt\n")

#Running the program
main()

List of products loaded successfully

=== SUMMARY (Console Output) ===
Total Products Processed: 5
Average Discount Applied: 12.80%
Report saved as: pricing_report.txt



In [6]:
# EXERCISE 2: Advance Server Log Analyser

import re #For pattern matching
import logging #Create log files
from datetime import datetime
from collections import defaultdict, Counter #Smart dictionaries that count things automatically

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler('analysis_audit.log'),
        logging.StreamHandler()
    ]
)

class LogAnalysisAnalyzer:
    def __init__(self, log_file):
        self.log_file = log_file
        # New regex for Python logging format: "2025-11-20 12:01:01,112 - INFO - Message"
        self.log_pattern = re.compile(
            r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) - (\w+) - (.+)'
        )
        
        # Security patterns for your log format
        self.security_patterns = {
            'brute_force': re.compile(r'Brute force attempt from (\S+) - (\d+) failed attempts'),
            'forbidden': re.compile(r'Forbidden access attempt: (\S+) -> (\S+)'),
            'sql_injection': re.compile(r'Potential SQL injection: (\S+) -> (\S+)'),
            'error': re.compile(r'Line (\d+): Error processing - (.+)')
        }

        # Statistics
        self.total_lines = 0
        self.log_levels = Counter()
        self.security_incidents = []
        self.brute_force_ips = defaultdict(int)
        self.forbidden_access = []
        self.sql_injections = []
        self.errors = []

    def parse_log_entry(self, line):
        """Parse Python logging format lines"""
        match = self.log_pattern.match(line)
        if not match:
            return None

        try:
            timestamp, level, message = match.groups()
            
            return {
                'timestamp': timestamp,
                'level': level,
                'message': message
            }
        except Exception as e:
            logging.debug(f"Error parsing line: {e}")
            return None

    def analyze_security_patterns(self, entry):
        """Analyze security patterns in the log messages"""
        message = entry['message']
        
        # Check for brute force attempts
        brute_match = self.security_patterns['brute_force'].search(message)
        if brute_match:
            ip, attempts = brute_match.groups()
            self.brute_force_ips[ip] = max(self.brute_force_ips[ip], int(attempts))
            incident = f"Brute force: {ip} - {attempts} attempts"
            self.security_incidents.append(incident)
            logging.warning(f"SECURITY: {incident}")

        # Check for forbidden access
        forbidden_match = self.security_patterns['forbidden'].search(message)
        if forbidden_match:
            ip, resource = forbidden_match.groups()
            incident = f"Forbidden: {ip} -> {resource}"
            self.forbidden_access.append(incident)
            self.security_incidents.append(incident)
            logging.info(f"FORBIDDEN: {incident}")

        # Check for SQL injection
        sql_match = self.security_patterns['sql_injection'].search(message)
        if sql_match:
            ip, url = sql_match.groups()
            incident = f"SQL Injection: {ip} -> {url}"
            self.sql_injections.append(incident)
            self.security_incidents.append(incident)
            logging.warning(f"SQL_INJECTION: {incident}")

        # Check for errors
        error_match = self.security_patterns['error'].search(message)
        if error_match:
            line_num, error_msg = error_match.groups()
            self.errors.append(f"Line {line_num}: {error_msg}")

    def analyze_log_file(self):
        try:
            logging.info(f"Beginning analysis of: {self.log_file}")

            with open(self.log_file, 'r', encoding='utf-8') as file:
                for line_number, raw_line in enumerate(file, 1):
                    line = raw_line.strip()
                    if not line:
                        continue

                    log_entry = self.parse_log_entry(line)
                    if not log_entry:
                        continue

                    self.total_lines += 1
                    self.log_levels[log_entry['level']] += 1
                    
                    # Analyze security patterns
                    self.analyze_security_patterns(log_entry)

                    if line_number % 50 == 0:
                        logging.info(f"Progress: {line_number} lines processed")
                
            logging.info(f"Analysis completed: {self.total_lines} total lines processed")
            
        except FileNotFoundError:
            logging.error(f"File not found: {self.log_file}")
            raise
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            raise

    def create_summary_report(self):
        try: 
            with open('summary_report.txt', 'w', encoding='utf-8') as f:
                f.write("LOG ANALYSIS SUMMARY REPORT\n")
                f.write("=" * 60 + "\n\n")
                    
                f.write("OVERVIEW\n")
                f.write("-" * 40 + "\n")                
                f.write(f"Total Lines Processed: {self.total_lines}\n")
                f.write(f"Log File Analyzed: {self.log_file}\n\n")
                    
                f.write("LOG LEVEL DISTRIBUTION\n")
                f.write("-" * 40 + "\n")
                for level, count in self.log_levels.most_common():
                    percentage = (count / self.total_lines) * 100
                    f.write(f"{level:>8}: {count:>4} ({percentage:5.1f}%)\n")
                f.write("\n")
                    
                f.write("SECURITY SUMMARY\n")
                f.write("-" * 40 + "\n")
                f.write(f"Total Security Incidents: {len(self.security_incidents)}\n")
                f.write(f"Brute Force Attacks: {len(self.brute_force_ips)}\n")
                f.write(f"Forbidden Access Attempts: {len(self.forbidden_access)}\n")
                f.write(f"SQL Injection Attempts: {len(self.sql_injections)}\n")
                f.write(f"Processing Errors: {len(self.errors)}\n\n")
                
                f.write("BRUTE FORCE ATTACKS DETAIL\n")
                f.write("-" * 40 + "\n")
                for ip, attempts in self.brute_force_ips.items():
                    f.write(f"IP: {ip} - Max attempts: {attempts}\n")
                f.write("\n")
                    
            logging.info("Summary report created: summary_report.txt")
                
        except Exception as e:
            logging.error(f"Failed to create summary: {e}")

    def create_security_report(self):
        try: 
            with open('security_incidents.txt', 'w', encoding='utf-8') as f:
                f.write("SECURITY INCIDENTS DETAILED REPORT\n")
                f.write("=" * 60 + "\n\n")
                    
                f.write("ALL SECURITY INCIDENTS\n")
                f.write("-" * 40 + "\n")
                for incident in self.security_incidents:
                    f.write(f"{incident}\n")
                f.write("\n")
                    
                f.write("PROCESSING ERRORS\n")
                f.write("-" * 40 + "\n")
                for error in self.errors:
                    f.write(f"{error}\n")
                    
            logging.info("Security report created: security_incidents.txt")
                
        except Exception as e:
            logging.error(f"Failed to create security report: {e}")

def main():
    print("Starting Log Analysis Analyzer...")
    
    analyzer = LogAnalysisAnalyzer('example_log.txt')  # Use your actual file
    
    try:
        # Analyze the log file
        analyzer.analyze_log_file()
        
        # Generate reports
        analyzer.create_summary_report()
        analyzer.create_security_report()
        
        # Show results
        print("\n" + "="*50)
        print("ANALYSIS COMPLETED SUCCESSFULLY!")
        print("="*50)
        print(f"Lines processed: {analyzer.total_lines}")
        print(f"Security incidents: {len(analyzer.security_incidents)}")
        print(f"Brute force attacks: {len(analyzer.brute_force_ips)}")
        print(f"SQL injection attempts: {len(analyzer.sql_injections)}")
        print(f"Processing errors: {len(analyzer.errors)}")
        print("\nGenerated reports:")
        print("• summary_report.txt - Analysis statistics")
        print("• security_incidents.txt - Security findings") 
        print("• analysis_audit.log - Analysis process log")
        print("="*50)
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")

if __name__ == "__main__":
    main()

2025-11-30 20:11:30,330 - INFO - Beginning analysis of: example_log.txt
2025-11-30 20:11:30,334 - INFO - FORBIDDEN: Forbidden: 192.168.1.44 -> /admin
2025-11-30 20:11:30,336 - WARNING - SQL_INJECTION: SQL Injection: 88.23.91.14 -> /login?user=admin'--
2025-11-30 20:11:30,337 - WARNING - SECURITY: Brute force: 10.0.0.15 - 3 attempts
2025-11-30 20:11:30,339 - INFO - FORBIDDEN: Forbidden: 10.0.0.22 -> /etc/passwd
2025-11-30 20:11:30,341 - WARNING - SQL_INJECTION: SQL Injection: 193.14.55.21 -> /products?id=4
2025-11-30 20:11:30,342 - INFO - FORBIDDEN: Forbidden: 172.16.8.77 -> /root
2025-11-30 20:11:30,343 - WARNING - SECURITY: Brute force: 10.0.0.15 - 4 attempts
2025-11-30 20:11:30,344 - WARNING - SQL_INJECTION: SQL Injection: 66.92.13.5 -> /search?q=DROP
2025-11-30 20:11:30,347 - INFO - FORBIDDEN: Forbidden: 212.88.99.18 -> /admin/config
2025-11-30 20:11:30,348 - WARNING - SECURITY: Brute force: 172.20.10.5 - 3 attempts
2025-11-30 20:11:30,350 - INFO - FORBIDDEN: Forbidden: 8.8.8.8 -> /

Starting Log Analysis Analyzer...

ANALYSIS COMPLETED SUCCESSFULLY!
Lines processed: 68
Security incidents: 61
Brute force attacks: 9
SQL injection attempts: 20
Processing errors: 5

Generated reports:
• summary_report.txt - Analysis statistics
• security_incidents.txt - Security findings
• analysis_audit.log - Analysis process log
